<a href="https://colab.research.google.com/github/MichaelHMorei/arquivos_de_apoio/blob/main/Igtipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

In [32]:
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [33]:
os.environ["SPARK_HOME"]

'/content/spark-3.2.1-bin-hadoop2.7'

In [34]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [35]:
pip install spark-nlp==2.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 15.8 MB/s 


In [36]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.2.1


In [56]:
import pandas as pd
import numpy as np
import os
import zipfile
import requests
from io import BytesIO
#Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark
from pyspark.sql.types import *
from datetime import *
from pyspark import SparkConf, SparkContext

In [18]:
#Site contendo os dados
url = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"

In [13]:
#Criando pasta para armazenar os dados
os.makedirs('./enadade2020',exist_ok = True)

In [19]:
#Fazendo o download do conteudo
file =  BytesIO(
     requests.get(url).content)
 
# Extraindo o conteudo do arquivo zip na pasta criada.
zip = zipfile.ZipFile(file)
zip.extractall("./enadade2020")

In [48]:
df_M = spark.read.option("encoding", "cp1252").csv("./enadade2020/DADOS/MICRODADOS_ENEM_2020.csv",inferSchema = 'True', sep =";", header ='True') 

In [85]:
df_M.show(10)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [72]:
df_M.createOrReplaceTempView('df_enada')

In [94]:
retur = spark.sql("""
  select df_enada.SG_UF_PROVA, df_enada.NU_NOTA_MT , df_enada.NU_NOTA_LC from df_enada
    where df_enada.SG_UF_PROVA = 'MG'
""")


In [108]:
retur.select('SG_UF_PROVA','NU_NOTA_MT').filter(col('NU_NOTA_MT') > 0).groupBy("SG_UF_PROVA").agg(avg("NU_NOTA_MT")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         MG|550.5736538835149|
+-----------+-----------------+



In [110]:
retur.select('SG_UF_PROVA','NU_NOTA_LC').filter(col('NU_NOTA_LC') > 0).groupBy("SG_UF_PROVA").agg(mean("NU_NOTA_LC")).show()

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_LC)|
+-----------+-----------------+
|         MG|540.4511216344497|
+-----------+-----------------+



In [107]:
retur.filter(col('NU_NOTA_LC') > 0).show()

+-----------+----------+----------+
|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_LC|
+-----------+----------+----------+
|         MG|     582.1|     586.4|
|         MG|     536.4|     514.6|
|         MG|     504.3|     568.7|
|         MG|     637.8|     619.0|
|         MG|     491.1|     491.7|
|         MG|     648.6|     579.5|
|         MG|     555.1|     503.5|
|         MG|     641.6|     604.6|
|         MG|      null|     575.1|
|         MG|     452.3|     544.3|
|         MG|     509.8|     450.5|
|         MG|     637.2|     617.0|
|         MG|     355.2|     402.0|
|         MG|     555.5|     577.3|
|         MG|     452.5|     613.4|
|         MG|     438.5|     419.7|
|         MG|     389.9|     421.2|
|         MG|     399.4|     506.6|
|         MG|     685.5|     662.4|
|         MG|     531.0|     576.6|
+-----------+----------+----------+
only showing top 20 rows

